# Calculation of the Feasible Joint Reaction Loads

The importance of evaluating the feasible muscle forces is demonstrated in the
context of joint reaction analysis. An accurate estimation of the muscle forces
is essential for the assessment of joint reaction loads. Consequently, the null
space muscle forces can significantly alter the reaction forces without
affecting the movement. The process is separated into four steps:

1. Perform an inverse kinematics (IK), static optimization (SO) and joint
reaction analysis (JRA) using OpenSim to generate the required data for the next
steps.

2. Calculate the feasible muscle forces that satisfy both the action and the
physiological constraints of the muscles.

3. For each distinct muscle force realization perform a joint reaction analysis.

4. Post-process the reaction loads.

In [2]:
# notebook general configuration
%load_ext autoreload
%autoreload 2

import os
import pickle
import numpy as np
import sympy as sp
from IPython.display import display
sp.interactive.printing.init_printing()
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 6.0)
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Subject Data

Please provide the following information for the subject under study:

- **.osim** the scaled OpenSim model (after scaling the model using a static
    pose)

- **.trc** the marker trajectories of the movement (based on OpenSim file
    requirements)

- **.mot** the ground reaction forces (based on the OpenSim file format)

- **.xml** describing how to apply the ground reaction forces (based on OpenSim
  force requirements)

- **.xml** file with the reserve actuator for the SO (based on OpenSim requirements)

- **subject mass** (used for scaling the reaction loads, and is not a
    requirement)

- **heel strike** and **toe off** events (used for annotation of the results,
  and are not so important)


The following calculations are performed for the *Gait10dof18musc* dataset. 

In [3]:
# subject data
mass = 72.6  # kg
g = 9.8  # m/s^2
body_weight = mass * g

# time events of toe off and heel strike
heel_strike_right = [0.65, 1.85]
toe_off_right = [0.15, 1.4]
heel_strike_left = [0.0, 1.25]
toe_off_left = [0.8, 2]

# required files and directories
subject_dir = os.getcwd() + '/../dataset/Gait10dof18musc/'
model_file = subject_dir + 'subject01.osim'
trc_file = subject_dir + 'subject01_walk.trc'
grf_file = subject_dir + 'subject01_walk_grf.mot'
grf_xml_file = subject_dir + 'subject01_walk_grf.xml'
reserve_actuators_file = subject_dir + 'reserve_actuators.xml'
results_dir = subject_dir + 'notebook_results/'
feasible_set_dir = results_dir + 'feasible_force_set/'
jra_results_dir = results_dir + 'joint_reaction_analyses/'
figures_dir = results_dir + 'fig/'

if not (os.path.isfile(model_file) and
        os.path.isfile(trc_file) and
        os.path.isfile(grf_file) and
        os.path.isfile(grf_xml_file) and
        os.path.isfile(reserve_actuators_file)):
    raise RuntimeError('required files do not exist')
    
if not (os.path.isdir(results_dir) and 
        os.path.isdir(feasible_set_dir) and
        os.path.isdir(jra_results_dir) and
        os.path.isdir(figures_dir)):
    raise RuntimeError('required folders do not exist')

## Step 1: Perform Required OpenSim Analyses

In order to perform the feasible muscle force calculations, the kinematics and
kinetics that satisfy the experimental measured motion and ground reaction
forces must be calculated. Furthermore, OpenSim JRA is performed so as to
compare the feasible reaction loads in step 4. The following functions make use
of the OpenSim IK, SO and RJA analyses.

In [ ]:
from opensim_utils import perform_ik, perform_so, perform_jra, plot_sto

# perform OpenSim inverse kinematics
ik_file = perform_ik(model_file, trc_file, results_dir)
plot_sto(ik_file, 4, save=True)

# perform OpenSim static optimization
(so_force_file, so_activation_file) = perform_so(model_file, ik_file, grf_file, 
                                                 grf_xml_file, reserve_actuators_file, 
                                                 results_dir)
plot_sto(so_force_file, 4, save=True)

# perform OpenSim joint reaction analysis
jra_file = perform_jra(model_file, ik_file, grf_file, grf_xml_file,
                       reserve_actuators_file, so_force_file, results_dir)

# store file names so that they can be loaded without running this section
pickle.dump((ik_file, so_force_file, jra_file), file(results_dir + 'opensim_files.dat', 'w'))

In [4]:
# get file names from step 1
(ik_file, so_force_file, jra_file) = pickle.load(file(results_dir + 'opensim_files.dat', 'r'))

## Step 2: Calculate the Feasible Muscle Forces

The feasible muscle forces are calculated below. Initially, the moment arm and
maximum muscle force quantities are computed for each instance of the
movement. Then the following inequality is formed assuming a linear muscle model

\begin{equation}\label{equ:linear-muscle-null-space-inequality}
  \begin{gathered}
    f_m = f_{max} \circ a_m = f_m^{\parallel} +
    N_{R} f_{m0},\; 0 \preceq a_m \preceq 1
    \rightarrow \\
    \begin{bmatrix}
      - N_{R} \\
      \hdashline
      N_{R}
    \end{bmatrix}
    f_{m0} \preceq
    \begin{bmatrix}
      f_m^{\parallel} \\
      \hdashline
      f_{max} - f_m^{\parallel}
    \end{bmatrix} \\
    Z f_{m0} \preceq \beta
  \end{gathered}
\end{equation}

where $a_m \in \Re^{m}$ represents a vector of muscle activations, $f_{max} \in
\Re^{m}$ a vector specifying the maximum muscle forces, $\circ$ the Hadamard
(elementwise) product, $f_m^{\parallel}$ the particular muscle force solution
that satisfies the action, $N_{R}$ the moment arm null space and $f_{m0}$ the
null space forces.

In [ ]:
# since this section of code may be called independently after step 1
# ensure that cells 2, 5 and 9 are evaluated
from IPython.display import Javascript
Javascript("Jupyter.notebook.execute_cells([2, 5, 9])")

In [ ]:
from opensim_utils import calculate_muscle_data
from util import null_space, construct_muscle_space_inequality, \
    convex_bounded_vertex_enumeration, readMotionFile, \
    index_containing_substring, write_as_sto_file
    
# calculate the moment arm and maximum muscle forces assuming a linear muscle
# model (in a future version this can calculate the properties for a nonlinear
# muscle model)
moment_arm, max_force = calculate_muscle_data(model_file, ik_file)

# read SO results
so_header, so_labels, so_data = readMotionFile(so_force_file)
so_data = np.array(so_data)
coordinates = moment_arm[0].shape[0]
muscles = moment_arm[0].shape[1]
time = so_data[:, 0]
entries = time.shape[0]

# collect quantities for computing the feasible muscle forces
NR = []
Z = []
b = []
fm_par = []
for t in tqdm(range(0, entries)):
    # get tau, R, Fmax
    fm = so_data[t, 1:(muscles + 1)]  # time the first column
    RT_temp = moment_arm[t, :, :]
    fmax_temp = max_force[t, :]

    # calculate the reduced rank (independent columns) null space to avoid
    # singularities
    NR_temp = null_space(RT_temp)
    
    # fm_par = fm is used instead of fm_par = -RBarT * tau because the
    # muscle may not be able to satisfy the action. In OpenSim reserve
    # actuators are used to ensure that Static Optimization can satisfy the
    # action. In this case, we ignore the residual forces and assume that fm
    # is the minimum effort solution. If the model is able to satisfy the
    # action without needing reserve forces then we can use fm_par = -RBarT
    # * tau as obtained form Inverse Dynamics.
    fm_par_temp = fm

    Z_temp, b_temp = construct_muscle_space_inequality(NR_temp,
                                                       fm_par_temp,
                                                       fmax_temp)

    # append results
    NR.append(NR_temp)
    Z.append(Z_temp)
    b.append(b_temp)
    fm_par.append(fm_par_temp)

The next step is to sample the inequality $Z f_{m0} \leq \beta$. This is the
bottleneck of the analysis. The *convex_bounded_vertex_enumeration* uses the
lsr method, which is a vertex enumeration algorithm for finding the vertices
of a polytope in $O(v m^3)$.

In [ ]:
# calculate the feasible muscle force set by sampling the inequality
f_set = []
for t in tqdm(range(0, entries)):
    try:
        fs = convex_bounded_vertex_enumeration(Z[t], b[t][:, 0], 0, method='lrs')
    except:
        print('inequlity is infeasible thus append previous iteration')
        f_set.append(f_set[-1])
        continue
        
    temp = []
    for i in range(0, fs.shape[0]):
        temp.append(fm_par[t] + NR[t].dot(fs[i, :]))

    f_set.append(temp)

# serialization f_set -> [time x feasible force set set x muscles] so as to
# avoid recomputing it
pickle.dump(f_set, file(feasible_set_dir + 'f_set.dat', 'w'))

Finally, store the feasible muscle force set into multiple .sto files that can
be used by the JRA.

In [ ]:
# keep only muscle forces
idx = index_containing_substring(so_labels, 'FX')[0]
labels = so_labels[:idx]

# find largest feasible set
S = len(f_set[0])
for fs in f_set:
    if len(fs) > S:
        S = len(fs)

# export muscle force realizations
for j in tqdm(range(0, S)):
    data_temp = []
    for i in range(0, so_data.shape[0]):
        data_temp.append(f_set[i][j % len(f_set[i])])

    write_as_sto_file(feasible_set_dir + str(j) + '.sto', labels,
                      time, np.array(data_temp))

## Step 3: Perform Joint Reaction Analyses on Feasible Muscle Forces

For each distinct muscle force realization computed in the previous step
initiate a JRA so as to calculate the influence of the muscle forces on the
joint reaction loads. Due to memory leaks in OpenSim JRA implementation this
process must be restarted since it can cause problems with the RAM usage. The if
statement is used so as to continue from the last performed analysis.

In [ ]:
# since this section of code may be called independently after step 2
# ensure that cells 2, 5 and 9 are evaluated
from IPython.display import Javascript
Javascript("Jupyter.notebook.execute_cells([2, 5, 9])")

In [ ]:
from opensim_utils import perform_jra

# get all files in the directory
feasible_force_files = os.listdir(feasible_set_dir)

# remove files that are not .sto
feasible_force_files = [e for e in feasible_force_files if '.sto' in e]

# perform joint reaction analyses
previous_iteration = 0
for i, force_file in enumerate(tqdm(feasible_force_files)):
    # change the previous_iteration variable
    if i < previous_iteration:
         continue

    if i > previous_iteration + 200:
        print('please shutdown this notebook and reopen (RAM usage problem)')
        print('set previous_iteration=' + str(previous_iteration + 200))
        break
        
    perform_jra(model_file, ik_file, grf_file, grf_xml_file,
                reserve_actuators_file, feasible_set_dir + force_file,
                jra_results_dir, prefix=str(i) + '_')

## Step 4: Postprocess Joint Reaction Loads

This section collects the joint reaction loads calculated in the previous step.

In [1]:
# since this section of code may be called independently after step 3
# ensure that cells 2, 5 and 9 are evaluated
from IPython.display import Javascript
Javascript("Jupyter.notebook.execute_cells([2, 5, 9])")

<IPython.core.display.Javascript object>

In [5]:
from util import readMotionFile, index_containing_substring

# select the joint of interest
joint = 'hip_r'

# load OpenSim JRA results (step 1)
os_header, os_labels, os_data = readMotionFile(jra_file)
os_data = np.array(os_data)
joint_index = index_containing_substring(os_labels, joint)
assert(joint_index != -1)

# get all files from the JRA batch simulation (step 3)
jra_files = os.listdir(jra_results_dir)

# remove files that are not joint reactions
jra_files = [e for e in jra_files if 'ReactionLoads' in e]
        
# allocate the necessary space to collect all results
solutions_to_keep = len(jra_files)
simulationData = np.empty([solutions_to_keep,
                           os_data.shape[0],
                           os_data.shape[1]],
                          dtype=float)

# collect reaction loads
for i, f in enumerate(tqdm(jra_files)):
    header, labels, data = readMotionFile(jra_results_dir + f)
    simulationData[i, :, :] = np.array(data)

100%|██████████| 16524/16524 [01:59<00:00, 138.28it/s]


The following section of code compares the feasible and OpenSim reaction loads
for the joint of interest. The reaction loads are normalized using the body
weight of the subject. The heel strike and toe off events are annotated
accordingly.

In [6]:
# select the heel strike and toe off events
if '_l' in joint:
    heel_strike = heel_strike_left
    toe_off = toe_off_left
else:
    heel_strike = heel_strike_right
    toe_off = toe_off_right

# plot data min/max reactions vs OpenSim JRA
joints = 3
fig, ax = plt.subplots(nrows=1, ncols=joints, figsize=(15, 5))
for i in range(0, joints):
    # plot feasible reaction loads
    min_reaction = np.min(simulationData[1:, 1:, joint_index[i]] / body_weight, 
                          axis=0)
    max_reaction = np.max(simulationData[1:, 1:, joint_index[i]] / body_weight, 
                          axis=0)
    ax[i].fill_between(os_data[1:, 0], min_reaction, max_reaction, 
                       color='b', alpha=0.2, label='Feasible Reactions')
    # plot OpenSim reaction loads
    ax[i].plot(os_data[1:, 0], os_data[1:, joint_index[i]] / body_weight,
               '-.r', label='OpenSim JRA')
    # annotate the heel strike and toe off regions
    min_min = np.min(min_reaction)
    max_max = np.max(max_reaction)
    ax[i].vlines(x=heel_strike, ymin=min_min, ymax=max_max,
                 color='c', linestyle='--', label='HS')
    ax[i].vlines(x=toe_off, ymin=min_min, ymax=max_max,
                 color='m', linestyle=':', label='TO')
    # figure settings
    ax[i].set_title(os_labels[joint_index[i]])
    ax[i].set_xlabel('time (s)')
    ax[i].set_ylabel('reaction / body weight')
    if i == joints - 1:
        ax[i].legend()

# export results
fig.tight_layout()
fig.savefig(figures_dir + joint + '.pdf',
            format='pdf', dpi=300)
fig.savefig(figures_dir + joint + '.png',
            format='png', dpi=300)
# fig.show()

We observe that the results obtained from OpenSim joint reaction analysis
predict low reaction load levels, since the minimum muscle effort criterion is
used to compute the muscle forces, ignoring muscle co-contraction. On the
contrary, it is possible to calculate the feasible reactions without making any
prior assumption, which can limit the scope and extent of the analysis. Last and
perhaps most importantly, the large range of possible values confirms that
misinterpretation of the results is possible if the null space solutions are
ignored.